# Nevada Procurement: Anomaly Detection Analysis

**Objective**: Identify potential anomalies, outliers, and data quality issues in Nevada procurement data using statistical methods.

**Data**: Nevada procurement contracts and purchase orders silver data

**Key Methods**:
- Statistical outlier detection (Z-score analysis)
- Benford's Law testing for fraud detection
- Data integrity checks (date validity, amount consistency)
- Split purchase analysis

**Coverage**: Contracts (49.8%), Purchase Orders (5.2%) - use with appropriate caveats

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
from scipy import stats
from scipy.stats import chisquare
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)

print("Setup complete")

In [ ]:
# Load data for anomaly detection
conn = duckdb.connect()

# Load contracts data
contracts_query = """
SELECT 
    contract_id,
    vendor_name,
    organization,
    fiscal_year_begin,
    dollars_spent_to_date,
    begin_date,
    end_date,
    contract_duration_days,
    daily_spend_rate,
    is_zero_spend,
    contract_status
FROM read_parquet("../../data/silver/contracts/version=v0.3.0/*/data.parquet")
WHERE contract_id IS NOT NULL
"""

contracts = conn.execute(contracts_query).df()
print(f"Loaded {len(contracts):,} contract records")
print(f"Zero spend contracts: {contracts['is_zero_spend'].sum()} ({contracts['is_zero_spend'].mean()*100:.1f}%)")
print(f"Total spend: ${contracts['dollars_spent_to_date'].sum():,.2f}")

# Load purchase orders data
pos_query = """
SELECT 
    po_id,
    vendor_name,
    organization,
    fiscal_year,
    sent_date,
    total_amount,
    revision_number,
    status_category
FROM read_parquet("../../data/silver/purchase_orders/version=v0.5.0/*/data.parquet")
WHERE po_id IS NOT NULL
"""

pos = conn.execute(pos_query).df()
print(f"\nLoaded {len(pos):,} purchase order records")
print(f"Total PO amount: ${pos['total_amount'].sum():,.2f}")
print(f"Revisions: {(pos['revision_number'] > 0).sum()} ({(pos['revision_number'] > 0).mean()*100:.1f}%)")

contracts.head()

In [ ]:
# Statistical Outlier Detection
print("=== STATISTICAL OUTLIER ANALYSIS ===")

# Contract amount outliers (exclude zero spend)
non_zero_contracts = contracts[contracts['dollars_spent_to_date'] > 0]
if len(non_zero_contracts) > 0:
    amounts = non_zero_contracts['dollars_spent_to_date']
    z_scores = np.abs(stats.zscore(amounts))
    outliers = non_zero_contracts[z_scores > 3]
    
    print(f"Contract amount outliers (|z| > 3): {len(outliers)} / {len(non_zero_contracts)} ({len(outliers)/len(non_zero_contracts)*100:.1f}%)")
    
    if len(outliers) > 0:
        print("\nTop Amount Outliers:")
        outlier_cols = ['contract_id', 'vendor_name', 'organization', 'dollars_spent_to_date']
        top_outliers = outliers.nlargest(5, 'dollars_spent_to_date')[outlier_cols]
        for _, row in top_outliers.iterrows():
            print(f"  {row['contract_id']}: ${row['dollars_spent_to_date']:,.2f} ({row['vendor_name'][:30]}...)")

# Duration outliers
duration_data = contracts[contracts['contract_duration_days'] > 0]
if len(duration_data) > 0:
    duration_z = np.abs(stats.zscore(duration_data['contract_duration_days']))
    duration_outliers = duration_data[duration_z > 3]
    
    print(f"\nDuration outliers: {len(duration_outliers)} contracts")
    if len(duration_outliers) > 0:
        print(f"Longest contract: {duration_outliers['contract_duration_days'].max()} days")
        print(f"Shortest outlier: {duration_outliers['contract_duration_days'].min()} days")

# Daily spend rate outliers
daily_spend_data = contracts[contracts['daily_spend_rate'] > 0]
if len(daily_spend_data) > 0:
    daily_z = np.abs(stats.zscore(daily_spend_data['daily_spend_rate']))
    daily_outliers = daily_spend_data[daily_z > 3]
    
    print(f"\nDaily spend rate outliers: {len(daily_outliers)} contracts")
    if len(daily_outliers) > 0:
        print(f"Highest daily rate: ${daily_outliers['daily_spend_rate'].max():,.2f}/day")

print(f"\nOverall outlier summary:")
print(f"Amount outliers: {len(outliers) if len(non_zero_contracts) > 0 else 0}")
print(f"Duration outliers: {len(duration_outliers) if len(duration_data) > 0 else 0}")
print(f"Daily rate outliers: {len(daily_outliers) if len(daily_spend_data) > 0 else 0}")

In [ ]:
# Benford's Law Analysis for Fraud Detection
def benford_test(amounts, name="Dataset"):
    """Test if amounts follow Benford's Law distribution"""
    # Filter positive amounts only
    amounts = amounts[amounts > 0]
    
    if len(amounts) < 100:
        return {"name": name, "n": len(amounts), "warning": "Sample too small for Benford test"}
    
    # Extract first digits
    first_digits = amounts.astype(str).str[0].astype(int)
    
    # Observed frequencies
    observed_counts = first_digits.value_counts().reindex(range(1, 10), fill_value=0)
    observed = observed_counts / len(amounts)
    
    # Expected Benford frequencies
    expected = np.array([np.log10(1 + 1/d) for d in range(1, 10)])
    
    # Ensure observed and expected sum to same value for chi-square test
    expected_counts = expected * len(amounts)
    
    # Chi-square test with proper normalization
    try:
        chi_stat, p_value = chisquare(observed_counts, expected_counts)
    except ValueError:
        # If normalization issues, use manual calculation
        chi_stat = sum((observed_counts - expected_counts)**2 / expected_counts)
        from scipy.stats import chi2
        p_value = 1 - chi2.cdf(chi_stat, df=8)  # 8 degrees of freedom (9 digits - 1)
    
    return {
        "name": name,
        "n": len(amounts),
        "chi_stat": chi_stat,
        "p_value": p_value,
        "anomaly_flag": p_value < 0.05,
        "observed": observed,
        "expected": expected
    }

print("=== BENFORD'S LAW ANALYSIS ===")

# Test contract amounts
contract_benford = benford_test(contracts['dollars_spent_to_date'], "Contract Amounts")
print(f"Contract amounts (n={contract_benford['n']})")
if 'warning' in contract_benford:
    print(f"  Warning: {contract_benford['warning']}")
else:
    print(f"  Chi-square: {contract_benford['chi_stat']:.3f}, p-value: {contract_benford['p_value']:.3f}")
    print(f"  Anomaly flag: {contract_benford['anomaly_flag']} (p < 0.05 indicates potential manipulation)")

# Test PO amounts
po_benford = benford_test(pos['total_amount'], "Purchase Order Amounts")
print(f"\nPurchase order amounts (n={po_benford['n']})")
if 'warning' in po_benford:
    print(f"  Warning: {po_benford['warning']}")
else:
    print(f"  Chi-square: {po_benford['chi_stat']:.3f}, p-value: {po_benford['p_value']:.3f}")
    print(f"  Anomaly flag: {po_benford['anomaly_flag']}")

# Visualize Benford distribution if enough data
if 'observed' in contract_benford:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Contract amounts
    digits = range(1, 10)
    ax1.bar(digits, contract_benford['observed'], alpha=0.7, label='Observed')
    ax1.plot(digits, contract_benford['expected'], 'ro-', label='Benford Expected')
    ax1.set_xlabel('First Digit')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Contract Amounts vs Benford\'s Law')
    ax1.legend()
    ax1.set_xticks(digits)
    
    # PO amounts if enough data
    if 'observed' in po_benford:
        ax2.bar(digits, po_benford['observed'], alpha=0.7, label='Observed')
        ax2.plot(digits, po_benford['expected'], 'ro-', label='Benford Expected')
        ax2.set_xlabel('First Digit')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Purchase Order Amounts vs Benford\'s Law')
        ax2.legend()
        ax2.set_xticks(digits)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Data Integrity Checks
print("=== DATA INTEGRITY ANALYSIS ===")

# Date validity checks
date_issues = contracts[
    (contracts['begin_date'].notna() & contracts['end_date'].notna()) &
    (contracts['end_date'] < contracts['begin_date'])
]
print(f"Contracts with end_date < begin_date: {len(date_issues)}")

# Negative amounts
negative_contracts = contracts[contracts['dollars_spent_to_date'] < 0]
negative_pos = pos[pos['total_amount'] < 0]
print(f"Contracts with negative spend: {len(negative_contracts)}")
print(f"POs with negative amounts: {len(negative_pos)}")

# Duration consistency
duration_check = contracts[
    (contracts['begin_date'].notna() & contracts['end_date'].notna() & 
     contracts['contract_duration_days'].notna())
]
if len(duration_check) > 0:
    duration_check = duration_check.copy()
    duration_check['calculated_duration'] = (duration_check['end_date'] - duration_check['begin_date']).dt.days
    duration_check['duration_diff'] = abs(duration_check['calculated_duration'] - duration_check['contract_duration_days'])
    
    duration_mismatches = duration_check[duration_check['duration_diff'] > 5]  # Allow 5-day tolerance
    print(f"Duration mismatches (>5 days): {len(duration_mismatches)} / {len(duration_check)} ({len(duration_mismatches)/len(duration_check)*100:.1f}%)")

# Missing critical fields
print(f"\nMissing data analysis:")
print(f"Contracts missing vendor_name: {contracts['vendor_name'].isna().sum()}")
print(f"Contracts missing organization: {contracts['organization'].isna().sum()}")
print(f"POs missing vendor_name: {pos['vendor_name'].isna().sum()}")
print(f"POs missing sent_date: {pos['sent_date'].isna().sum()}")

In [ ]:
# Split Purchase Detection
print("=== SPLIT PURCHASE ANALYSIS ===")

# Look for multiple POs to same vendor on same day with similar amounts
split_query = """
WITH same_day_vendor AS (
    SELECT 
        vendor_name,
        organization,
        sent_date,
        COUNT(*) as po_count,
        SUM(total_amount) as total_amount,
        AVG(total_amount) as avg_amount,
        STDDEV(total_amount) as stddev_amount,
        MIN(total_amount) as min_amount,
        MAX(total_amount) as max_amount
    FROM read_parquet("../../data/silver/purchase_orders/version=v0.5.0/*/data.parquet")
    WHERE vendor_name IS NOT NULL 
        AND sent_date IS NOT NULL
        AND total_amount > 0
    GROUP BY vendor_name, organization, sent_date
    HAVING COUNT(*) > 1
)
SELECT *,
    CASE 
        WHEN stddev_amount / avg_amount < 0.1 THEN 'Similar amounts'
        WHEN max_amount / min_amount < 2.0 THEN 'Close amounts' 
        ELSE 'Varied amounts'
    END as amount_pattern
FROM same_day_vendor
WHERE po_count >= 2
ORDER BY po_count DESC, total_amount DESC
"""

split_results = conn.execute(split_query).df()
print(f"Same-day, same-vendor PO groups: {len(split_results)}")

if len(split_results) > 0:
    # Flag potential split purchases
    potential_splits = split_results[
        (split_results['amount_pattern'].isin(['Similar amounts', 'Close amounts'])) &
        (split_results['po_count'] >= 2)
    ]
    
    print(f"Potential split purchases: {len(potential_splits)}")
    if len(potential_splits) > 0:
        print("\nTop Potential Split Purchase Patterns:")
        split_cols = ['vendor_name', 'organization', 'sent_date', 'po_count', 'total_amount', 'amount_pattern']
        print(potential_splits[split_cols].head().to_string(index=False))

# Weekend/holiday PO analysis
pos_with_dates = pos[pos['sent_date'].notna()].copy()
if len(pos_with_dates) > 0:
    pos_with_dates['weekday'] = pos_with_dates['sent_date'].dt.dayofweek
    weekend_pos = pos_with_dates[pos_with_dates['weekday'].isin([5, 6])]  # Saturday, Sunday
    
    print(f"\nWeekend POs: {len(weekend_pos)} / {len(pos_with_dates)} ({len(weekend_pos)/len(pos_with_dates)*100:.1f}%)")
    if len(weekend_pos) > 0:
        print(f"Weekend PO total: ${weekend_pos['total_amount'].sum():,.2f}")

In [ ]:
# Create anomaly visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Nevada Procurement: Anomaly Detection Analysis', fontsize=16, y=0.98)

# 1. Amount distribution with outliers
if len(non_zero_contracts) > 0:
    log_amounts = np.log10(non_zero_contracts['dollars_spent_to_date'])
    axes[0, 0].hist(log_amounts, bins=30, alpha=0.7, edgecolor='black')
    if len(outliers) > 0:
        outlier_log = np.log10(outliers['dollars_spent_to_date'])
        axes[0, 0].hist(outlier_log, bins=30, alpha=0.8, color='red', edgecolor='black', label=f'Outliers (n={len(outliers)})')
        axes[0, 0].legend()
    axes[0, 0].set_xlabel('Log10(Contract Amount)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Contract Amount Distribution (Log Scale)')

# 2. Daily spend rate vs contract amount
if len(daily_spend_data) > 0:
    axes[0, 1].scatter(daily_spend_data['dollars_spent_to_date'], 
                      daily_spend_data['daily_spend_rate'], alpha=0.6)
    if len(daily_outliers) > 0:
        axes[0, 1].scatter(daily_outliers['dollars_spent_to_date'], 
                          daily_outliers['daily_spend_rate'], 
                          color='red', s=50, alpha=0.8, label=f'Outliers (n={len(daily_outliers)})')
        axes[0, 1].legend()
    axes[0, 1].set_xlabel('Total Contract Amount')
    axes[0, 1].set_ylabel('Daily Spend Rate')
    axes[0, 1].set_title('Daily Spend Rate vs Contract Amount')
    axes[0, 1].set_xscale('log')
    axes[0, 1].set_yscale('log')

# 3. PO revision patterns
if len(pos) > 0:
    revision_counts = pos['revision_number'].value_counts().sort_index()
    revision_counts.plot(kind='bar', ax=axes[1, 0], alpha=0.7)
    axes[1, 0].set_xlabel('Revision Number')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].set_title('Purchase Order Revision Distribution')
    axes[1, 0].tick_params(axis='x', rotation=0)

# 4. Temporal patterns (POs by day of week)
if len(pos_with_dates) > 0:
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    daily_counts = pos_with_dates['weekday'].value_counts().sort_index()
    daily_counts.index = [day_names[i] for i in daily_counts.index]
    daily_counts.plot(kind='bar', ax=axes[1, 1], alpha=0.7)
    axes[1, 1].set_xlabel('Day of Week')
    axes[1, 1].set_ylabel('PO Count')
    axes[1, 1].set_title('Purchase Orders by Day of Week')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Save the figure
plt.savefig('../output/anomaly_detection_analysis.png', dpi=300, bbox_inches='tight')
print("Charts saved to ../output/anomaly_detection_analysis.png")

In [ ]:
# Export anomaly detection results
if len(non_zero_contracts) > 0 and len(outliers) > 0:
    outliers.to_csv('../output/amount_outliers.csv', index=False)

if len(split_results) > 0:
    split_results.to_csv('../output/potential_split_purchases.csv', index=False)

if len(date_issues) > 0:
    date_issues.to_csv('../output/date_integrity_issues.csv', index=False)

# Anomaly summary
anomaly_summary = {
    'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
    'contracts_analyzed': len(contracts),
    'pos_analyzed': len(pos),
    'amount_outliers': len(outliers) if len(non_zero_contracts) > 0 else 0,
    'duration_outliers': len(duration_outliers) if len(duration_data) > 0 else 0,
    'daily_rate_outliers': len(daily_outliers) if len(daily_spend_data) > 0 else 0,
    'date_integrity_issues': len(date_issues),
    'potential_split_purchases': len(potential_splits) if len(split_results) > 0 else 0,
    'weekend_pos': len(weekend_pos) if len(pos_with_dates) > 0 else 0,
    'contract_benford_pvalue': contract_benford.get('p_value'),
    'contract_benford_anomaly': contract_benford.get('anomaly_flag'),
    'po_benford_pvalue': po_benford.get('p_value'),
    'po_benford_anomaly': po_benford.get('anomaly_flag')
}

pd.Series(anomaly_summary).to_csv('../output/anomaly_summary.csv')

print("Anomaly detection results exported:")
print("  - amount_outliers.csv")
print("  - potential_split_purchases.csv")
print("  - date_integrity_issues.csv")
print("  - anomaly_summary.csv")
print("  - anomaly_detection_analysis.png")

## Key Findings & Audit Implications

**Statistical Outliers**: [To be filled after running analysis]

**Benford's Law Results**: Tests for potential data manipulation in amount distributions

**Data Quality Issues**: Integrity checks reveal data consistency problems

**Audit Recommendations**:
- Investigate high-value outliers for approval compliance
- Review potential split purchases for threshold avoidance
- Validate weekend/holiday transactions for policy compliance
- Address data integrity issues in source systems

**Methodology Notes**:
- Z-score threshold of 3 for outlier detection (99.7% confidence)
- Benford's Law requires >100 observations for reliability
- Split purchase detection uses same-day, same-vendor heuristics
- Coverage limitations: PO analysis (5.2%) has high variance